File to train on the toy_df.csv (that came from the 001_Forbes Dataset)

In [64]:
import os
import ast
import torch
import random
import pandas as pd
from transformers import TapasConfig, TapasForQuestionAnswering, TapasTokenizer, AdamW

In [65]:
# load in tokenizer
tokenizer = TapasTokenizer.from_pretrained("google/tapas-base-finetuned-wtq")

In [66]:
# Function to convert the string representation to a list of tuples
def parse_answer_coords(coords_str):
    try:
        # Safely evaluate the string to a Python object
        coords = ast.literal_eval(coords_str)
        
        # Ensure the result is a list of tuples with integers
        if isinstance(coords, list) and all(
            isinstance(coord, (tuple, list)) and len(coord) == 2 and all(isinstance(x, int) for x in coord)
            for coord in coords
        ):
            return [tuple(coord) for coord in coords]  # Convert lists to tuples if needed
        else:
            raise ValueError(f"Invalid format for answer_coords: {coords_str}")
    except (ValueError, SyntaxError) as e:
        raise ValueError(f"Error parsing answer_coords: {coords_str}. Details: {e}")

In [67]:
# class to tokenizer dataset
class TableDataset(torch.utils.data.Dataset):
    def __init__(self, data, tokenizer):
        self.data = self.process_answer_coords_column(data)
        self.tokenizer = tokenizer
        self.table_csv_path = 'data/'

    def __getitem__(self, idx):
        item = self.data.iloc[idx]
        table = pd.read_csv(self.table_csv_path + item.dataset).astype(
            str
        )  # be sure to make your table data text only
        encoding = self.tokenizer(
            table=table,
            queries=item.question,
            answer_coordinates=item.answer_coords,
            answer_text=item.sample_answer,
            truncation=True,
            padding="max_length",
            return_tensors="pt",
        )
        # remove the batch dimension which the tokenizer adds by default
        encoding = {key: val.squeeze(0) for key, val in encoding.items()}
        # add the float_answer which is also required (weak supervision for aggregation case)
        encoding["float_answer"] = torch.tensor(item.float_answer)
        return encoding

    def __len__(self):
        return len(self.data)
    
    # change answer_coords from strings
    def process_answer_coords_column(self, data, column_name="answer_coords"):
        if column_name not in data.columns:
            raise ValueError(f"Column '{column_name}' does not exist in the dataset.")

        def parse_answer_coords(coords_str):
            try:
                coords = ast.literal_eval(coords_str)
                if isinstance(coords, list) and all(
                    isinstance(coord, (tuple, list)) and len(coord) == 2 and all(isinstance(x, int) for x in coord)
                    for coord in coords
                ):
                    return [tuple(coord) for coord in coords]
                else:
                    raise ValueError(f"Invalid format for answer_coords: {coords_str}")
            except (ValueError, SyntaxError) as e:
                raise ValueError(f"Error parsing answer_coords: {coords_str}. Details: {e}")
        data[column_name] = data[column_name].apply(parse_answer_coords)
        return data

In [68]:
# load in toy_df, tokenize and place in dataloader
csv_path = 'data/toy_df.csv'
 
toy_df = pd.read_csv(csv_path)
datasets = list(set(toy_df['dataset']))
train_datasets = datasets[:len(datasets)-2]
dev_datasets = datasets[len(datasets)-2:]
print('Dev Datasets:', dev_datasets)
print('Train Datasets:', train_datasets)
# Apply the function to the answer_coords column

# split into train and dev
toy_df_train = toy_df[toy_df['dataset'].isin(train_datasets)]
toy_df_dev = toy_df[toy_df['dataset'].isin(dev_datasets)]

# load train dataloader
train_dataset = TableDataset(toy_df_train, tokenizer)
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=16)

# load dev dataloader
dev_dataset = TableDataset(toy_df_dev, tokenizer)
dev_dataloader = torch.utils.data.DataLoader(dev_dataset, batch_size=16)

Dev Datasets: ['019_Aircraft.csv', '001_Forbes.csv']
Train Datasets: ['013_Roller.csv', '004_Taxi.csv', '012_Heart.csv', '017_Hacker.csv', '009_Central.csv', '018_Staff.csv', '015_Food.csv', '010_ECommerce.csv', '005_NYC.csv', '016_Holiday.csv', '008_Tornados.csv', '007_Fifa.csv', '011_SF.csv', '002_Titanic.csv']


/var/folders/f1/_zcz3_s54kz_1zxxwgs414pw0000gn/T/ipykernel_5245/1796820079.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[column_name] = data[column_name].apply(parse_answer_coords)


In [69]:
# load in WTQ model (from TAPAS documentation)
config = TapasConfig(
    num_aggregation_labels=4,
    use_answer_as_supervision=True,
    answer_loss_cutoff=0.664694,
    cell_selection_preference=0.207951,
    huber_loss_delta=0.121194,
    init_cell_selection_weights_to_zero=True,
    select_one_column=True,
    allow_empty_column_selection=False,
    temperature=0.0352513,
)
model_wtq = TapasForQuestionAnswering.from_pretrained("google/tapas-base-finetuned-wtq", config=config) # leave as is
model_finetun = TapasForQuestionAnswering.from_pretrained("google/tapas-base-finetuned-wtq", config=config) # finetune
optimizer = AdamW(model.parameters(), lr=5e-5)

/opt/anaconda3/envs/cs375/lib/python3.11/site-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [70]:
# finetune the WTQ model
model_finetune.train()
for epoch in range(20):
    print('On epoch:', epoch) 
    for batch in train_dataloader:
        input_ids = batch["input_ids"]
        attention_mask = batch["attention_mask"]
        token_type_ids = batch["token_type_ids"]
        labels = batch["labels"]
        numeric_values = batch["numeric_values"]
        numeric_values_scale = batch["numeric_values_scale"]
        float_answer = batch["float_answer"]
        
        if isinstance(numeric_values, pd.Series):
            numeric_values = numeric_values.iloc[:] 
            
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model_finetune(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            labels=labels,
            numeric_values=numeric_values,
            numeric_values_scale=numeric_values_scale,
            float_answer=float_answer,
        )
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        
print('Done Training')

On epoch: 0


/opt/anaconda3/envs/cs375/lib/python3.11/site-packages/transformers/models/tapas/tokenization_tapas.py:2673: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  text = normalize_for_match(row[col_index].text)
/opt/anaconda3/envs/cs375/lib/python3.11/site-packages/transformers/models/tapas/tokenization_tapas.py:1472: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  cell = row[col_index]
/opt/anaconda3/envs/cs375/lib/python3.11/site-packages/transformers/models/tapas/tokenization_tapas.py:2673: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as

/opt/anaconda3/envs/cs375/lib/python3.11/site-packages/transformers/models/tapas/tokenization_tapas.py:2673: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  text = normalize_for_match(row[col_index].text)
/opt/anaconda3/envs/cs375/lib/python3.11/site-packages/transformers/models/tapas/tokenization_tapas.py:1472: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  cell = row[col_index]
/opt/anaconda3/envs/cs375/lib/python3.11/site-packages/transformers/models/tapas/tokenization_tapas.py:2673: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as

/opt/anaconda3/envs/cs375/lib/python3.11/site-packages/transformers/models/tapas/tokenization_tapas.py:2673: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  text = normalize_for_match(row[col_index].text)
/opt/anaconda3/envs/cs375/lib/python3.11/site-packages/transformers/models/tapas/tokenization_tapas.py:1472: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  cell = row[col_index]
/opt/anaconda3/envs/cs375/lib/python3.11/site-packages/transformers/models/tapas/tokenization_tapas.py:2673: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as

/opt/anaconda3/envs/cs375/lib/python3.11/site-packages/transformers/models/tapas/tokenization_tapas.py:1472: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  cell = row[col_index]
/opt/anaconda3/envs/cs375/lib/python3.11/site-packages/transformers/models/tapas/tokenization_tapas.py:2673: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  text = normalize_for_match(row[col_index].text)
/opt/anaconda3/envs/cs375/lib/python3.11/site-packages/transformers/models/tapas/tokenization_tapas.py:1472: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as

...

/opt/anaconda3/envs/cs375/lib/python3.11/site-packages/transformers/models/tapas/tokenization_tapas.py:2673: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  text = normalize_for_match(row[col_index].text)
/opt/anaconda3/envs/cs375/lib/python3.11/site-packages/transformers/models/tapas/tokenization_tapas.py:1472: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  cell = row[col_index]
/opt/anaconda3/envs/cs375/lib/python3.11/site-packages/transformers/models/tapas/tokenization_tapas.py:2673: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as

/opt/anaconda3/envs/cs375/lib/python3.11/site-packages/transformers/models/tapas/tokenization_tapas.py:2673: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  text = normalize_for_match(row[col_index].text)
/opt/anaconda3/envs/cs375/lib/python3.11/site-packages/transformers/models/tapas/tokenization_tapas.py:1472: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  cell = row[col_index]
/opt/anaconda3/envs/cs375/lib/python3.11/site-packages/transformers/models/tapas/tokenization_tapas.py:2673: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as

/opt/anaconda3/envs/cs375/lib/python3.11/site-packages/transformers/models/tapas/tokenization_tapas.py:2673: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  text = normalize_for_match(row[col_index].text)
/opt/anaconda3/envs/cs375/lib/python3.11/site-packages/transformers/models/tapas/tokenization_tapas.py:1472: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  cell = row[col_index]
/opt/anaconda3/envs/cs375/lib/python3.11/site-packages/transformers/models/tapas/tokenization_tapas.py:2673: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as

/opt/anaconda3/envs/cs375/lib/python3.11/site-packages/transformers/models/tapas/tokenization_tapas.py:2673: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  text = normalize_for_match(row[col_index].text)
/opt/anaconda3/envs/cs375/lib/python3.11/site-packages/transformers/models/tapas/tokenization_tapas.py:1472: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  cell = row[col_index]
/opt/anaconda3/envs/cs375/lib/python3.11/site-packages/transformers/models/tapas/tokenization_tapas.py:2673: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as

Done Training


In [71]:
# Inference on Finetuned 
num_none = 0
num_agg = 0
num_correct = 0
num_correct_none = 0
num_correct_agg = 0
total_questions = 0
for file in dev_datasets:
    print('#################### (Finetuned) dev on:', file,'####################')
    table = pd.read_csv(f'data/{file}').astype(str)
    dev_df = toy_df_dev[toy_df_dev['dataset'] == file]
    queries = list(dev_df['question'])
    inputs = tokenizer(
        table = table,
        queries = queries,
        padding="max_length",
        return_tensors="pt",
        truncation = True
    )

    outputs = model_finetune(**inputs)
    predicted_answer_coordinates, predicted_aggregation_indices = tokenizer.convert_logits_to_predictions(
        inputs, outputs.logits.detach(), outputs.logits_aggregation.detach()
    )

    # print results:
    id2aggregation = {0: "NONE", 1: "SUM", 2: "AVERAGE", 3: "COUNT"}
    aggregation_predictions_string = [id2aggregation[x] for x in predicted_aggregation_indices]

    answers = []
    for coordinates in predicted_answer_coordinates:
        if len(coordinates) == 1:
            # only a single cell:
            answers.append(table.iat[coordinates[0]])
        else:
            # multiple cells
            cell_values = []
            for coordinate in coordinates:
                cell_values.append(table.iat[coordinate])
            answers.append(", ".join(cell_values))

    i = 0
    real_answers = list(dev_df['sample_answer'])
    answer_type = list(dev_df['type'])
    for query, answer, predicted_agg in zip(queries, answers, aggregation_predictions_string):
        total_questions += 1 
        print(query)
        print('Indeded type: ', answer_type[i])
        if predicted_agg == "NONE":
            print("     Predicted answer: " + answer)
            print("     Real answer: ", real_answers[i])
            if str(answer) == str(real_answers[i]):
                num_correct += 1
                num_correct_none += 1
            i += 1
            num_none += 1
            
        else:
            print("     Predicted Cells: " + predicted_agg + " > " + answer)
            if predicted_agg == 'COUNT':
                count = answer.split(',')
                answer = len(list(set(count)))
                print('     Predicted Count: ', answer)
                
            if predicted_agg == 'AVERAGE':
                total = 0
                a = answer.split(',')
                try:
                    a = [float(elem) for elem in a]
                except:
                    print('     Predicted Average: Error')
                    answer = None
                    continue
                num = len(a)
                answer = 0
                for elem in a:
                    answer += (elem/num)  
                print('     Predicted Average:', answer)
            if predicted_agg == 'SUM':
                s = answer.split(',')
                s = [float(elem) for elem in s]
                answer = 0
                for elem in s:
                    answer += elem
                print('     Predicted Sum:', answer)
                    
            print("     Real answer: ", str(real_answers[i]))
            if str(answer) == str(real_answers[i]):
                    num_correct += 1
                    num_correct_agg += 1
            i += 1
            num_agg += 1
            
print("")
print('Number of Questions: ', total_questions)
print('Overall Accuracy: ', num_correct/total_questions)
try:
    print('Number of NONE:', num_none)
    print('NONE Accuracy:', num_correct_none/num_none)
except:
    print('***** No questions with predicted Aggregation Tag NONE ***** ')
try:
    print('Number of Agg:', num_agg)
    print('Agg Accuracy:', num_correct_agg/num_agg)
except:
    print('***** No questions with predicted Aggregation Tag SUM, AVERAGE, or COUNT ***** ')
print("")

#################### (Finetuned) dev on: 019_Aircraft.csv ####################


/opt/anaconda3/envs/cs375/lib/python3.11/site-packages/transformers/models/tapas/tokenization_tapas.py:2673: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  text = normalize_for_match(row[col_index].text)
/opt/anaconda3/envs/cs375/lib/python3.11/site-packages/transformers/models/tapas/tokenization_tapas.py:1472: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  cell = row[col_index]


How many unique aircraft models are in the dataset?
Indeded type:  number
     Predicted Cells: COUNT > En route (ENR), En route (ENR), Unknown (UNK), Landing (LDG), Unknown (UNK), Landing (LDG), Approach (APR), Landing (LDG), Standing (STD), Approach (APR), Unknown (UNK), En route (ENR), Unknown (UNK), Pushback / towing (PBT), Landing (LDG), Landing (LDG), Landing (LDG), En route (ENR), Initial climb (ICL), Maneuvering (MNV)
     Predicted Count:  9
     Real answer:  20
What's the highest number of occupants recorded in an incident?
Indeded type:  number
     Predicted Cells: COUNT > 1, 17, 44, 1
     Predicted Count:  4
     Real answer:  86
How many incidents resulted in non-zero fatalities?
Indeded type:  number
     Predicted Cells: COUNT > Dahra-Wareho..., Carolina Aer..., Mount Banahao, Titograd Air..., near San Diego-No..., unknown, Bembridge Ai..., Seoul-Incheo..., near Guryev, near Valencia
     Predicted Count:  10
     Real answer:  10
Which aircraft model was involved in 

/opt/anaconda3/envs/cs375/lib/python3.11/site-packages/transformers/models/tapas/tokenization_tapas.py:2673: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  text = normalize_for_match(row[col_index].text)
/opt/anaconda3/envs/cs375/lib/python3.11/site-packages/transformers/models/tapas/tokenization_tapas.py:1472: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  cell = row[col_index]


What is the age of the youngest billionaire?
Indeded type:  number
     Predicted answer: 32.0
     Real answer:  32.0
What's the rank of the wealthiest non-self-made billionaire?
Indeded type:  number
     Predicted Cells: AVERAGE > 2448
     Predicted Average: 2448.0
     Real answer:  288
Which category does the richest billionaire belong to?
Indeded type:  category
     Predicted answer: 
     Real answer:  Food & Beverage
What's the country of origin of the oldest billionaire?
Indeded type:  category
     Predicted answer: United States
     Real answer:  United Kingdom
What's the gender of the billionaire with the highest philanthropy score?
Indeded type:  category
     Predicted answer: M
     Real answer:  M
What's the source of wealth for the youngest billionaire?
Indeded type:  category
     Predicted answer: fintech
     Real answer:  fintech

Number of Questions:  13
Overall Accuracy:  0.3076923076923077
Number of NONE: 9
NONE Accuracy: 0.3333333333333333
Number of Agg: 4
A

In [72]:
# Inference on WTQ 
num_none = 0
num_agg = 0
num_correct = 0
num_correct_none = 0
num_correct_agg = 0
total_questions = 0
for file in dev_datasets:
    print('#################### (WTQ) dev on:', file,'####################')
    table = pd.read_csv(f'data/{file}').astype(str)
    dev_df = toy_df_dev[toy_df_dev['dataset'] == file]
    queries = list(dev_df['question'])
    inputs = tokenizer(
        table = table,
        queries = queries,
        padding="max_length",
        return_tensors="pt",
        truncation = True
    )

    outputs = model_wtq(**inputs)
    predicted_answer_coordinates, predicted_aggregation_indices = tokenizer.convert_logits_to_predictions(
        inputs, outputs.logits.detach(), outputs.logits_aggregation.detach()
    )

    # print results:
    id2aggregation = {0: "NONE", 1: "SUM", 2: "AVERAGE", 3: "COUNT"}
    aggregation_predictions_string = [id2aggregation[x] for x in predicted_aggregation_indices]

    answers = []
    for coordinates in predicted_answer_coordinates:
        if len(coordinates) == 1:
            # only a single cell:
            answers.append(table.iat[coordinates[0]])
        else:
            # multiple cells
            cell_values = []
            for coordinate in coordinates:
                cell_values.append(table.iat[coordinate])
            answers.append(", ".join(cell_values))

    i = 0
    real_answers = list(dev_df['sample_answer'])
    answer_type = list(dev_df['type'])
    for query, answer, predicted_agg in zip(queries, answers, aggregation_predictions_string):
        total_questions += 1 
        print(query)
        print('Indeded type: ', answer_type[i])
        if predicted_agg == "NONE":
            print("     Predicted answer: " + answer)
            print("     Real answer: ", real_answers[i])
            if str(answer) == str(real_answers[i]):
                num_correct += 1
                num_correct_none += 1
            i += 1
            num_none += 1
            
        else:
            print("     Predicted Cells: " + predicted_agg + " > " + answer)
            if predicted_agg == 'COUNT':
                count = answer.split(',')
                answer = len(list(set(count)))
                print('     Predicted Count: ', answer)
                
            if predicted_agg == 'AVERAGE':
                total = 0
                a = answer.split(',')
                try:
                    a = [float(elem) for elem in a]
                except:
                    print('     Predicted Average: Error')
                    answer = None
                    continue
                num = len(a)
                answer = 0
                for elem in a:
                    answer += (elem/num)  
                print('     Predicted Average:', answer)
            if predicted_agg == 'SUM':
                s = answer.split(',')
                s = [float(elem) for elem in s]
                answer = 0
                for elem in s:
                    answer += elem
                print('     Predicted Sum:', answer)
                    
            print("     Real answer: ", str(real_answers[i]))
            if str(answer) == str(real_answers[i]):
                    num_correct += 1
                    num_correct_agg += 1
            i += 1
            num_agg += 1
            
print("")
print('Number of Questions: ', total_questions)
print('Overall Accuracy: ', num_correct/total_questions)
try:
    print('Number of NONE:', num_none)
    print('NONE Accuracy:', num_correct_none/num_none)
except:
    print('***** No questions with predicted Aggregation Tag NONE ***** ')
try:
    print('Number of Agg:', num_agg)
    print('Agg Accuracy:', num_correct_agg/num_agg)
except:
    print('***** No questions with predicted Aggregation Tag SUM, AVERAGE, or COUNT ***** ')
print("")

#################### (WTQ) dev on: 019_Aircraft.csv ####################


/opt/anaconda3/envs/cs375/lib/python3.11/site-packages/transformers/models/tapas/tokenization_tapas.py:2673: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  text = normalize_for_match(row[col_index].text)
/opt/anaconda3/envs/cs375/lib/python3.11/site-packages/transformers/models/tapas/tokenization_tapas.py:1472: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  cell = row[col_index]


How many unique aircraft models are in the dataset?
Indeded type:  number
     Predicted Cells: COUNT > En route (ENR), En route (ENR), Unknown (UNK), Landing (LDG), Unknown (UNK), Landing (LDG), Approach (APR), Landing (LDG), Standing (STD), Approach (APR), Unknown (UNK), En route (ENR), Unknown (UNK), Pushback / towing (PBT), Landing (LDG), Landing (LDG), Landing (LDG), En route (ENR), Initial climb (ICL), Maneuvering (MNV)
     Predicted Count:  9
     Real answer:  20
What's the highest number of occupants recorded in an incident?
Indeded type:  number
     Predicted Cells: COUNT > Fatalities: 1 / Occupants: 2, Fatalities: 5 / Occupants: 8, Fatalities: 8 / Occupants: 9, Fatalities: 44 / Occupants: 44, Fatalities: 1 / Occupants:, Fatalities: 0 / Occupants: 0, Fatalities: 0 / Occupants: 6
     Predicted Count:  7
     Real answer:  86
How many incidents resulted in non-zero fatalities?
Indeded type:  number
     Predicted Cells: COUNT > Dahra-Wareho..., Carolina Aer..., Mount Banahao

/opt/anaconda3/envs/cs375/lib/python3.11/site-packages/transformers/models/tapas/tokenization_tapas.py:2673: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  text = normalize_for_match(row[col_index].text)
/opt/anaconda3/envs/cs375/lib/python3.11/site-packages/transformers/models/tapas/tokenization_tapas.py:1472: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  cell = row[col_index]


What is the age of the youngest billionaire?
Indeded type:  number
     Predicted answer: 32.0
     Real answer:  32.0
What's the rank of the wealthiest non-self-made billionaire?
Indeded type:  number
     Predicted Cells: AVERAGE > 2448, 2448
     Predicted Average: 2448.0
     Real answer:  288
Which category does the richest billionaire belong to?
Indeded type:  category
     Predicted answer: Diversified
     Real answer:  Food & Beverage
What's the country of origin of the oldest billionaire?
Indeded type:  category
     Predicted answer: United States
     Real answer:  United Kingdom
What's the gender of the billionaire with the highest philanthropy score?
Indeded type:  category
     Predicted answer: M
     Real answer:  M
What's the source of wealth for the youngest billionaire?
Indeded type:  category
     Predicted answer: fintech
     Real answer:  fintech

Number of Questions:  13
Overall Accuracy:  0.3076923076923077
Number of NONE: 9
NONE Accuracy: 0.3333333333333333
N